Base에서 실행

# Textcuboid


## 1) Dataload NC data

In [1]:
import json
import pandas as pd
import numpy as np
import re
import string
import nltk


data=[]
with open('C:/Users/user/Desktop/bilm-tf-master/textdataset/News_Category_Dataset_v3.json', 'r') as f:
    for line in f:
        content = json.loads(line)
        data.append(content)
df = pd.DataFrame(data=data)

In [2]:
df = df[~df['short_description'].apply(lambda x: len(x)==0)]
print(df.shape)

#중복되는 카테고리 통합
df["category"] = df.category.replace(
    {
        "THE WORLDPOST": "WORLD NEWS",
        "WORLDPOST": "WORLD NEWS",
        "ARTS": "ARTS & CULTURE",
        "CULTURE & ARTS": "ARTS & CULTURE",
        "HEALTHY LIVING": "WELLNESS",
        "QUEER VOICES": "VOICES",
        "LATINO VOICES": "VOICES",
        "BLACK VOICES": "VOICES",
        "STYLE": "STYLE & BEATUY",
        "GREEN": "ENVIRONMENT",
        "TASTE": "FOOD & DRINK",
        "MONEY": "BUSINESS",
        "PARENTING": "PARENTS"
    }
)

df = df[['headline', 'short_description','category']]
df['input_data']= df.apply(lambda x: str(x['headline']) + str(x['short_description']), axis=1)

def clean_text(text):
    text=str(text).lower() #Converts text to lowercase
    text=re.sub('\d+', '', text) #removes numbers
    text=re.sub('\[.*?\]', '', text) #removes HTML tags
    text=re.sub('https?://\S+|www\.\S+', '', text) #removes url
    text=re.sub(r"["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", "", text) #removes emojis
    text=re.sub('[%s]' % re.escape(string.punctuation),'',text) #removes punctuations
    #text = re.sub('\n', '', text)
    #text = re.sub('\w*\d\w*', '', text)
    return text

df['clean_text']=df['input_data'].apply(clean_text)

class0=list(df[df['category'] == 'POLITICS'].sample(1000, random_state=1)['clean_text'])
class1=list(df[df['category'] == 'WELLNESS'].sample(1000, random_state=1)['clean_text'])
class2=list(df[df['category'] == 'ENTERTAINMENT'].sample(1000, random_state=1)['clean_text'])
class3=list(df[df['category'] == 'PARENTS'].sample(1000, random_state=1)['clean_text'])
class4=list(df[df['category'] == 'VOICES'].sample(1000, random_state=1)['clean_text'])
class5=list(df[df['category'] == 'STYLE & BEAUTY'].sample(1000, random_state=1)['clean_text'])
class6=list(df[df['category'] == 'TRAVEL'].sample(1000, random_state=1)['clean_text'])
class7=list(df[df['category'] == 'FOOD & DRINK'].sample(1000, random_state=1)['clean_text'])
class8=list(df[df['category'] == 'WORLD NEWS'].sample(1000, random_state=1)['clean_text'])
class9=list(df[df['category'] == 'BUSINESS'].sample(1000, random_state=1)['clean_text'])

X_list=class0+class1+class2+class3+class4+class5+class6+class7+class8+class9

y=[]
for i in range(10):
    y+=[i]*1000

clr_x_data=[]
pattern = '[^a-z ]'
for sen in X_list:
    clr_x_data.append(re.sub(pattern, ' ', sen))

X_list=[]
for sen in clr_x_data:
    X_list.append(' '.join(sen.split()))

train_idx=[]
for i in range(10):
    train_idx+=[j+i*1000 for j in range(800)]

test_idx=[]
for i in range(10):
    test_idx+=[j+800+i*1000 for j in range(200)]

x_train=[]
y_train=[]
for i in train_idx:
    x_train.append(X_list[i])
    y_train.append(y[i])

x_test=[]
y_test=[]
for i in test_idx:
    x_test.append(X_list[i])
    y_test.append(y[i])

to_txt_filter=x_train+x_test
y=list(y_train)+list(y_test)

(189815, 6)


In [3]:
#불용어 불러오기
import os
os.chdir('..')
with open('./english_stopwords.txt', 'r', encoding='utf-8') as file:
    stopwords = [line.strip() for line in file]

In [4]:
stopwords=stopwords+['aaa', 'ab']

In [5]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(stop_words=stopwords)
X_dtm = vect.fit_transform(to_txt_filter)
X_dtm = X_dtm.toarray()
X_new = SelectKBest(chi2, k=10000).fit(X_dtm, y)
TorF = X_new.get_support()
TorF
import numpy as np
word_view=np.array(vect.get_feature_names())
sw=word_view[TorF]

C:\Users\user\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'daren', 'hadn', 'herse', 'himse', 'itse', 'mayn', 'mightn', 'mon', 'mustn', 'myse', 'needn', 'oughtn', 'shan'] not in stop_words.
  warnings.warn(
C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [6]:
#선별된 10000개 단어
list(sw)

['abandoned',
 'abaya',
 'abbas',
 'abbey',
 'abc',
 'abducted',
 'abdulaziz',
 'abe',
 'abilities',
 'ability',
 'ableism',
 'aboard',
 'abortion',
 'abortions',
 'abrams',
 'abs',
 'absence',
 'abu',
 'abundance',
 'abuse',
 'aca',
 'academy',
 'accept',
 'acceptance',
 'accepted',
 'accepting',
 'access',
 'accessories',
 'accessory',
 'accident',
 'accidental',
 'accidents',
 'acclaim',
 'acclaimed',
 'accommodate',
 'accomplish',
 'accords',
 'account',
 'accounting',
 'accounts',
 'accumulate',
 'accuracy',
 'accurate',
 'accusations',
 'accuse',
 'accused',
 'accuser',
 'accusers',
 'accuses',
 'accusing',
 'ache',
 'achieve',
 'achievement',
 'acid',
 'acknowledge',
 'acknowledged',
 'aclu',
 'acosta',
 'acquisition',
 'acres',
 'acta',
 'acted',
 'action',
 'actions',
 'active',
 'actively',
 'activism',
 'activist',
 'activists',
 'activities',
 'activity',
 'acton',
 'actor',
 'actors',
 'actress',
 'actresses',
 'acts',
 'acupuncture',
 'acute',
 'adam',
 'adams',
 'adapt',

In [7]:
feature_lst10000=sw

## 2) textcuboid 생성

In [8]:
doc_lst=[]
for sen in x_train:
    doc_lst.append(sen.split())

In [9]:
test_lst=[]
for sen in x_test:
    test_lst.append(sen.split())

In [10]:
#Train_data에서 문서가 갖고 있는 선별한 feauture의 수 확인
count_lst=[]
for i in range(8000):
    total_feature_cnt=0
    for j in range(10000):
        if feature_lst10000[j] in doc_lst[i]:
            total_feature_cnt+=1
    count_lst.append(total_feature_cnt)
    
print('Train_data에서 가장 많은 feature를 가진 문서의 경우 feature',max(count_lst),' 개를 가짐')
print('Train_data에서 가장 적은 feature를 가진 문서의 경우 feature',min(count_lst),' 개를 가짐')

Train_data에서 가장 많은 feature를 가진 문서의 경우 feature 68  개를 가짐
Train_data에서 가장 적은 feature를 가진 문서의 경우 feature 1  개를 가짐


In [11]:
#Test_data에서 문서가 갖고 있는 선별한 feauture의 수 확인
count_lst=[]
for i in range(2000):
    
    total_feature_cnt=0
    for j in range(10000):
        if feature_lst10000[j] in test_lst[i]:
            total_feature_cnt+=1
    count_lst.append(total_feature_cnt)
    
print('Test_data에서 가장 많은 feature를 가진 문서의 경우 feature',max(count_lst),' 개를 가짐')
print('Test_data에서 가장 적은 feature를 가진 문서의 경우 feature',min(count_lst),' 개를 가짐')

Test_data에서 가장 많은 feature를 가진 문서의 경우 feature 41  개를 가짐
Test_data에서 가장 적은 feature를 가진 문서의 경우 feature 1  개를 가짐


In [12]:
#1-Channel TextCuboid 생성
textcuboid=[]

for i in range(8000):
    frame1=np.zeros((68,768))
    idx_cnt=0
    for j in range(10000):
        if feature_lst10000[j] in doc_lst[i]:
            frame1[idx_cnt]=np.load('./bert_embedding/train(bert)/doc%d.npy'%i)[doc_lst[i].index(feature_lst10000[j])]
            idx_cnt+=1
    textcuboid.append(frame1)

textcuboid=np.array(textcuboid)

np.save('./1-Channel textcuboid_nc(bert).npy',textcuboid)

In [13]:
#1-Channel TextCuboid 생성
textcuboid_test=[]

for i in range(2000):
    frame1=np.zeros((68,768))
    idx_cnt=0
    for j in range(10000):
        if feature_lst10000[j] in test_lst[i]:
            frame1[idx_cnt]=np.load('./bert_embedding/test(bert)/test%d.npy'%i)[test_lst[i].index(feature_lst10000[j])]
            idx_cnt+=1

    textcuboid_test.append(frame1)

textcuboid_test=np.array(textcuboid_test)
np.save('./1-Channel textcuboid_test_nc(bert).npy',textcuboid_test)